In [146]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-california-wildfire-data/POSTFIRE_MASTER_DATA_SHARE_140463065990229786.geojson
/kaggle/input/the-california-wildfire-data/b8aeb030-140d-43d2-aa29-1a80862e3d62.csv
/kaggle/input/customer-churn-dataset/customer_churn_dataset-testing-master.csv
/kaggle/input/customer-churn-dataset/customer_churn_dataset-training-master.csv


In [147]:
# importing modules
import time
import csv
import numpy as np
import dask.dataframe as dd
import sqlite3
import pandas as pd
import pyarrow.csv as pv_csv
from io import StringIO
import warnings
warnings.filterwarnings("ignore")

# Set the seed for reproducibility
np.random.seed(42)  # For NumPy
import random
random.seed(42)  # For random module
import torch
torch.manual_seed(42)  # If using PyTorch (optional)


In [148]:
# Path for input data
path = "/kaggle/input/customer-churn-dataset/customer_churn_dataset-training-master.csv"

# creating  dictionary to store all execution times
execution_times = {}

In [149]:
# A view of dataframe is only shown here as it will not be shown further in the notebook
pd.read_csv(path,low_memory=False).head(2)

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.0,17.0,1.0
1,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.0,6.0,1.0


Note: Please run each execution twice to see the exact time as some modules take high time to run once and then the time is significantly reduced 

## 1. Using the csv Module (Built-in)

In [150]:
# Using csv.reader
start = time.time()
with open(path, 'r') as file:
    reader = csv.reader(file)
    data_csv_reader = list(reader)  # Read all rows into a list
execution_times["csv_reader"] = time.time() - start


In [151]:
# Using csv.DictReader
start = time.time()
with open(path, 'r') as file:
    reader = csv.DictReader(file)
    data_csv_dictreader = [row for row in reader]  # Convert to a list of dictionaries
execution_times["csv_dictreader"] = time.time() - start

## 2. Using pandas Library

In [152]:
start = time.time()
dd = pd.read_csv(path,low_memory=False)
execution_times["pandas"] = time.time() - start

## 3. csv.reader with io.StringIO

In [153]:
# Using csv.reader with io.StringIO
start = time.time()
file = StringIO(path)  # Convert string to file-like object
reader = csv.reader(file)
data_csv_stringio = list(reader)  # Read all rows into a list
execution_times["csv_reader_StringIO"] = time.time() - start

## Using sqlite3 (For Loading into a Database)

In [154]:

# Step 1: Create an SQLite database and insert the CSV data
conn = sqlite3.connect(':memory:')  # Use in-memory database for simplicity
cursor = conn.cursor()

# Read the CSV file into a pandas DataFrame
df_csv = pd.read_csv(path)

# Creating a table dynamically based on the CSV columns
columns = ', '.join([f"{col} TEXT" for col in df_csv.columns])  # Define columns as TEXT type
cursor.execute(f'''CREATE TABLE data ({columns})''')

# Insert data into the table
df_csv.to_sql('data', conn, if_exists='replace', index=False)


440833

In [155]:
# Step 2: Measuring execution time for loading data from SQLite
start = time.time()
# Query data from SQLite
df_sqlite = pd.read_sql_query("SELECT * FROM data", conn)
execution_times["sqlite3"] = time.time() - start

## Using dask for Large CSV Files

In [156]:
import dask.dataframe as dd
# Step 1: Measuring execution time for loading the data with Dask
start = time.time()

# Read the CSV file with Dask (this loads the data lazily)
df_dask = dd.read_csv(path,assume_missing=True)

# Triggering computation by forcing Dask to load the data into memory
df_dask.compute()

execution_times["dask"] = time.time() - start


## Using pyarrow for High-Performance Data Loading

In [157]:
# Step 1: Measure execution time for loading the data with PyArrow
start = time.time()

# Read the CSV file using PyArrow
table = pv_csv.read_csv(path)

# Convert to pandas DataFrame (if needed)
df_pyarrow = table.to_pandas()

execution_times["pyarrow"] = time.time() - start


## Using modin for Parallel Loading

In [158]:
# !pip install modin -q

In [159]:
import modin.pandas as mpd

# Step 1: Measure execution time for loading the data with Modin
start = time.time()

# Read the CSV file using Modin (which uses pandas API)
df_modin = mpd.read_csv(path)

execution_times["modin"] = time.time() - start

## Using petl for Transformations

In [160]:
# Install petl module
# !pip install petl -q

In [161]:
import petl as etl

# Step 1: Measure execution time for loading the data with Petl
start = time.time()
# Read the CSV file using Petl
table = etl.fromcsv(path)
execution_times["petl"] = time.time() - start


## Using pyspark for Big Data

In [162]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder.appName("BigDataCSVProcessing").getOrCreate()

# Step 1: Measure execution time for loading the data with PySpark
start = time.time()

# Load the CSV file using PySpark (returns a DataFrame)
df_spark = spark.read.csv(path, header=True, inferSchema=True)
execution_times["pyspark"] = time.time() - start


## Using Polars

In [163]:
# !pip install polars -q

In [164]:
import polars as pl

# Step 1: Measure execution time for loading the data with Polars
start = time.time()

# Read the CSV file using Polars
df_polars = pl.read_csv(path, ignore_errors=True)

execution_times["polars"] = time.time() - start


## Using DuckDB

In [165]:
import duckdb

# Step 1: Measure execution time for loading the data with DuckDB
start = time.time()

# Open a DuckDB connection and read the CSV file into a DuckDB relation (SQL table-like structure)
conn = duckdb.connect()
df_duckdb = conn.execute(f"SELECT * FROM read_csv_auto('{path}')").fetchdf()

execution_times["duckdb"] = time.time() - start

# Close the DuckDB connection
conn.close()


In [145]:
df_comparison = pd.DataFrame(list(execution_times.items()), columns=["Method", "Execution Time (s)"])
df_comparison["Execution Time (s)"] = df_comparison["Execution Time (s)"].apply(lambda x: f"{x:.3e}")  # Convert to scientific notation

# Display results
df_comparison

,Method,Execution Time (s)
0,csv_reader,1.648e+00
1,csv_dictreader,1.847e+00
2,pandas,5.103e-01
3,csv_reader_StringIO,1.643e-04
4,sqlite3,2.081e+00
5,dask,5.484e-01
6,pyarrow,2.936e-01
7,modin,4.102e-01
8,petl,1.388e-04
9,pyspark,1.194e+00
